In [1]:
import numpy as np
import cv2
from PIL import Image,ImageEnhance
from imutils.perspective import four_point_transform
import cv2
import os

In [19]:
def auto_canny(image, sigma=0.4):
	# compute the median of the single channel pixel intensities
	v = np.median(image)
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
	# return the edged image
	return edged
def top_view(pathImage):
	img_ori=Image.open(pathImage)
	contrast_enhancer = ImageEnhance.Contrast(img_ori)
	pil_enhanced_image = contrast_enhancer.enhance(2)
	enhanced_image = np.asarray(pil_enhanced_image)
	kernel = np.ones((5,5),np.uint8)
	
	enhanced_image = cv2.morphologyEx(enhanced_image, cv2.MORPH_CLOSE, kernel, iterations= 3)
	imgGray = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2GRAY)
	imgBlur = cv2.GaussianBlur(imgGray, (5, 5), 1)
	imgThreshold = auto_canny(imgBlur)
	#img = Image.fromarray(np.uint8(imgThreshold))
	#img.show()
	kernel = np.ones((5, 5))
	imgDial = cv2.dilate(imgThreshold, kernel, iterations=2)
	imgThreshold = cv2.erode(imgDial, kernel, iterations=1)
	img2 = Image.fromarray(np.uint8(imgThreshold))
	img2.show()
	contours, _ = cv2.findContours(imgThreshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
	contours = sorted(contours, key=cv2.contourArea, reverse=True)

	for contour in contours:
		peri = cv2.arcLength(contour, True)
		approx = cv2.approxPolyDP(contour, 0.05 * peri, True)
		if len(approx) == 4:
			doc_cnts = approx
			break
	image_oricoppy=np.asarray(Image.open(pathImage)).copy()
	warped = four_point_transform(image_oricoppy, doc_cnts.reshape(4, 2))
	img_return = Image.fromarray(np.uint8(warped))
	img_return = img_return.resize(size=(640, 640))
	img3 = Image.fromarray(np.uint8(warped))
	img3.show()
	return img_return
	

In [24]:
pathImage = r"omr\311311848_3297464403867300_2424470930712633274_n.jpg"
warped=top_view(pathImage)
warped.save('311311848_3297464403867300_2424470930712633274_n.jpeg')

In [43]:
import numpy as np
import torch
import pandas as pd
from uuid import uuid4

In [51]:
model = torch.hub.load('WongKinYiu/yolov7', 'custom','best_ch.pt')
model.eval()
#model.conf = 0.7
#model.iou = 0.7

pathImage=r"311113384_2508730335933542_4154020231467925786_n.jpeg"
pli_img =Image.open(pathImage)
cv_image = np.asarray(pli_img)
cv_image = cv2.resize(cv_image, (640, 640))
cv_image_ori=np.asarray(cv_image).copy()
cv_image = [cv_image]
results = model(cv_image, size=640)
df=results.pandas().xyxy[0]

for index, row in df.iterrows():
    #print(row["xmin"])
    x1=int(row["xmin"])
    x2=int(row["xmax"])
    y1=int(row["ymin"])
    y2=int(row["ymax"])
    cropped_image = cv_image_ori[y1:y2,x1:x2]
    rand_token = uuid4()
    Image.fromarray(np.uint8(cropped_image)).save(r"ch/"+str(rand_token)+".jpg")

Using cache found in C:\Users\Artit/.cache\torch\hub\WongKinYiu_yolov7_main

                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv             

Adding autoShape... 


YOLOR  2022-10-16 torch 1.12.1+cu113 CUDA:0 (NVIDIA GeForce GTX 950M, 4095.875MB)



In [38]:
df

,xmin,ymin,xmax,ymax,confidence,class,name
